In [1]:
%load_ext autoreload
%autoreload 2
import os
import gym
import neat
import pickle
import numpy as np
from gene import Gene
from simulator import Simulator
import matplotlib.pyplot as plt
from matplotlib import animation
from tf_neat.recurrent_net import RecurrentNet

np.set_printoptions(precision=10, suppress=True)


def make_net(genome, config, bs):
    return RecurrentNet.create(genome, config, bs)


def make_env():
    return gym.make("CartPole-v0")


def save_frames_as_gif(frames, path='./', filename='gym_animation.gif'):
    # Mess with this to change frame size
    plt.clf()
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi=72)

    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)
    anim.save(path + filename, writer='imagemagick', fps=60)


param = {
    "g_c": 2,
    "g_s": 3,
    "max_state": 5,
    "react_depth": 2,
    "neuron_num": 6,
    "input_num": 4,
    "output_num": 1
}

config_path = os.path.join(os.path.dirname(os.path.abspath('')), "lab/neat.cfg")
config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

filepath = f"test_case"
case_num = 5

savePath = f"{filepath}\\{case_num}"
genome = ""
with open(f"{savePath}\\genome.JSON", 'rb') as fr:
    genome = pickle.load(fr)

net = make_net(genome, config, 1)
gene = Gene(net, param)
gene.expression()
gene.model_display(savePath)
model = gene.model
simulator = Simulator(model, engine="phy", ode_language='FORTRAN')  # FORTRAN, PYTHON
if simulator.checksum == False : 
    print("model failed...")
    print(f"info : {simulator.model['n']} {simulator.model['e']}")
else :
    env = make_env()
    env.reset()
    frames = []
    input_vector = np.array([0] * param["input_num"])
    fitnesses = np.zeros(1)
    for epoch in range(300):
        frames.append(env.render(mode="rgb_array"))
        simulator.input(input_vector)
        success = simulator.run(1)
        if success == False:  done = True
        output_vector = simulator.output()
        action = 1 if output_vector[0] > 250 else 0
        # print(output_vector)
        state, reward, done, _ = env.step(action)
        # print(np.exp(state))
        input_vector = 10 * np.minimum(10, np.exp(state))
        fitnesses += reward

        if savePath and (done or (epoch + 1) % 10 == 0):
            simulator.visualize(savePath=savePath)
            plt.clf()

        if done: break

    env.close()
    save_frames_as_gif(frames, path=savePath + '\\')
    plt.clf()

MovieWriter imagemagick unavailable; using Pillow instead.


<Figure size 432x288 with 0 Axes>

<Figure size 600x400 with 0 Axes>

In [79]:
import numpy as np
a = np.arange(5)
print(a)
b=a[:]
print(b)
a[2] = 101
print(b)

[0 1 2 3 4]
[0 1 2 3 4]
[  0   1 101   3   4]


In [87]:
np.take(a,[2,3]).base

In [7]:
model["M_"].shape

(104, 898)

In [109]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
plt.clf()
# 200ms 간격으로 균일하게 샘플된 시간
g = np.array([5,
    10,
    15,
    20,
    25,
    30,
    35,
    40,
    45,
    50])

cartpole_avg = np.array([4.5,
    8,
    11.4,
    14,
    26,
    48,
    57,
    64,
    68,
    71])

cartpole_best = np.array([10,
    33,
    72,
    99,
    99,
    99,
    99,
    98,
    99,
    99])

# 빨간 대쉬, 파란 사각형, 녹색 삼각형


# Define x, y, and xnew to resample at.
x = g
y = cartpole_avg
xnew = np.linspace(5, 50, num=41, endpoint=True)

# Define interpolators.
f_linear = interp1d(x, y)
f_cubic = interp1d(x, y, kind='cubic')

plt.xlabel('generation')
plt.ylabel('fitness')
plt.grid(True, axis='y', alpha=0.5, linestyle='--')
plt.legend(['1','average fitness','2', 'best fitness'])

# Plot.

plt.plot(xnew, f_linear(xnew), '-', color='w')
plt.plot(xnew, f_cubic(xnew), '--', color='tab:red')
plt.plot(x, y, 's', color='tab:red', label='CartPole avg')


# Define x, y, and xnew to resample at.
x = g
y = cartpole_best
xnew = np.linspace(5, 50, num=41, endpoint=True)

# Define interpolators.
f_linear = interp1d(x, y)
f_cubic = interp1d(x, y, kind='cubic')

# Plot.
plt.plot(xnew, f_linear(xnew), '-', color='w')
plt.plot(xnew, f_cubic(xnew), '--', color='tab:red')
plt.plot(x, y, '^',color='tab:red', label='CartPole best')

g = np.array([5,
    10,
    15,
    20,
    25,
    30,
    35,
    40,
    45,
    50])

cartpole_avg = np.array([
    2.1,
2.3,
4.2,
4.1,
6.2,
7.6,
11.5,
23.6,
38.9,
45.3,
])

cartpole_best = np.array([
    5,
7,
11,
11,
22,
38,
86,
89,
89,
91,
])

# 빨간 대쉬, 파란 사각형, 녹색 삼각형


# Define x, y, and xnew to resample at.
x = g
y = cartpole_avg
xnew = np.linspace(5, 50, num=41, endpoint=True)

# Define interpolators.
f_linear = interp1d(x, y)
f_cubic = interp1d(x, y, kind='cubic')

plt.xlabel('generation')
plt.ylabel('fitness')
plt.grid(True, axis='y', alpha=0.5, linestyle='--')
plt.legend(['1','average fitness','2', 'best fitness'])

# Plot.

plt.plot(xnew, f_linear(xnew), '-', color='w')
plt.plot(xnew, f_cubic(xnew), '--', color='tab:blue')
plt.plot(x, y, 's', color='tab:blue', label='FrozenLake avg')


# Define x, y, and xnew to resample at.
x = g
y = cartpole_best
xnew = np.linspace(5, 50, num=41, endpoint=True)

# Define interpolators.
f_linear = interp1d(x, y)
f_cubic = interp1d(x, y, kind='cubic')

# Plot.
plt.plot(xnew, f_linear(xnew), '-', color='w')
plt.plot(xnew, f_cubic(xnew), '--', color='tab:blue')
plt.plot(x, y, '^',color='tab:blue', label='FrozenLake best')

plt.legend()
'''
plt.grid(True, axis='y', alpha=0.5, linestyle='--')
plt.legend(['average fitness', 'best fitness'])
plt.plot(g, cartpole_avg_smooth, color='tab:red')
plt.plot(g, cartpole_avg, color='tab:red', marker='s')
plt.plot(g, cartpole_best, color='tab:blue', marker='^')

plt.xlabel('generation')
plt.ylabel('fitness')
plt.legend(['average fitness', 'best fitness'])
#plt.savefig("../paper/source/test.png")
'''
plt.savefig("../paper/source/total.png")
#plt.show()


C:\Users\arizona\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [81]:
xnew

array([0.   , 0.225, 0.45 , 0.675, 0.9  , 1.125, 1.35 , 1.575, 1.8  ,
       2.025, 2.25 , 2.475, 2.7  , 2.925, 3.15 , 3.375, 3.6  , 3.825,
       4.05 , 4.275, 4.5  , 4.725, 4.95 , 5.175, 5.4  , 5.625, 5.85 ,
       6.075, 6.3  , 6.525, 6.75 , 6.975, 7.2  , 7.425, 7.65 , 7.875,
       8.1  , 8.325, 8.55 , 8.775, 9.   ])

In [121]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
plt.clf()
# 200ms 간격으로 균일하게 샘플된 시간
g = np.array([5,
    10,
    15,
    20,
    25,
    30,
    35,
    40,
    45,
    50])

cartpole_avg = np.array([
np.inf,
np.inf,
np.inf,
100,
100,
97.00996678,
96.34551495,
96.34551495,
95.34883721,
93.68770764,


])

cartpole_best = np.array([
np.inf,
np.inf,
np.inf,
np.inf,
np.inf,
np.inf,
np.inf,
100,
99.63144963,
98.4029484,
])

# 빨간 대쉬, 파란 사각형, 녹색 삼각형


# Define x, y, and xnew to resample at.
x = g
y = cartpole_avg
xnew = np.linspace(5, 50, num=41, endpoint=True)

# Define interpolators.
f_linear = interp1d(x, y)
f_cubic = interp1d(x, y, kind='cubic')

plt.xlabel('generation')
plt.ylabel('training time (%)')
plt.grid(True, axis='y', alpha=0.5, linestyle='--')
plt.legend(['1','average fitness','2', 'best fitness'])

# Plot.

plt.plot(xnew, f_linear(xnew), '--', color='tab:red')
plt.plot(xnew, f_cubic(xnew), '--', color='tab:red')
plt.plot(x, y, 's', color='tab:red', label='CartPole')


# Define x, y, and xnew to resample at.
x = g
y = cartpole_best
xnew = np.linspace(5, 50, num=41, endpoint=True)

# Define interpolators.
f_linear = interp1d(x, y)
f_cubic = interp1d(x, y, kind='cubic')

# Plot.
plt.plot(xnew, f_linear(xnew),'--', color='tab:blue')
plt.plot(xnew, f_cubic(xnew), '--', color='tab:blue')
plt.plot(x, y, '^',color='tab:blue', label='FrozenLake')

plt.legend()
'''
plt.grid(True, axis='y', alpha=0.5, linestyle='--')
plt.legend(['average fitness', 'best fitness'])
plt.plot(g, cartpole_avg_smooth, color='tab:red')
plt.plot(g, cartpole_avg, color='tab:red', marker='s')
plt.plot(g, cartpole_best, color='tab:blue', marker='^')

plt.xlabel('generation')
plt.ylabel('fitness')
plt.legend(['average fitness', 'best fitness'])
#plt.savefig("../paper/source/test.png")
'''
plt.savefig("../paper/source/training_time.png")
#plt.show()

C:\Users\arizona\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\arizona\Anaconda3\envs\tensorflow\lib\site-packages\scipy\interpolate\interpolate.py:623: RuntimeWarning: invalid value encountered in subtract
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\Users\arizona\Anaconda3\envs\tensorflow\lib\site-packages\scipy\interpolate\interpolate.py:626: RuntimeWarning: invalid value encountered in add
  y_new = slope*(x_new - x_lo)[:, None] + y_lo
C:\Users\arizona\Anaconda3\envs\tensorflow\lib\site-packages\scipy\interpolate\interpolate.py:623: RuntimeWarning: invalid value encountered in subtract
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\Users\arizona\Anaconda

In [33]:
import numpy as np
a = [
 [-1,  1,  0,  0],
 [-1,  0,  0 , 0],
 [ 1,  0,-1,  0],
 [ 0, 1,  0,  0],
 [ 0,  0,  1, -1],
 [ 0, -1,  0 , 1],
]

b = [
[1,  0,  1,  0,  1,  1],
[0,  1,  1 , 1,  1,  1],
]


In [34]:
M_ = np.array(a)
M = np.array(b)

In [35]:
np.dot(M, M_)

array([[0, 0, 0, 0],
       [0, 0, 0, 0]])

In [39]:

P, L, U = scipy.linalg.lu(M_)


print(P)

print(L)

print(U)

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]
[[ 1.  0.  0.  0.]
 [ 1.  1.  0.  0.]
 [-1. -1.  1.  0.]
 [-0. -0. -1.  1.]
 [-0. -1. -0. -0.]
 [-0.  1. -0. -1.]]
[[-1.  1.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0. -1.]]


In [40]:
np.dot(np.dot(P,L),U)

array([[-1.,  1.,  0.,  0.],
       [-1.,  0.,  0.,  0.],
       [ 1.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1., -1.],
       [ 0., -1.,  0.,  1.]])

In [47]:
np.dot(np.dot(M,P),L)

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [45]:

arr_inv = np.linalg.inv(P)

print(arr_inv)


[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]
